In [1]:
%%time
import re as charRemoval
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import time
import pickle
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import numpy as np
import pandas as pd
import csv

[nltk_data] Downloading package wordnet to /Users/nyuad/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/nyuad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/nyuad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


CPU times: user 603 ms, sys: 142 ms, total: 745 ms
Wall time: 1.17 s


In [2]:
class TweetsPreProcessing:
    def TweetsProcessing(self, tweetlists):
        Tweets_Processed=[]
        for t in tweetlists:
            Tweets_Processed.append((self.TweetCleaner(t)))
        return Tweets_Processed
    # tweet cleaner function convert texts to lowercases and removes hashtags urls and usernames
    def TweetCleaner(self, t):
        stopwordlist = list(stopwords.words('english'))
        urlLinks=r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
        username='@[^\s]+'
        Alpha_Patterns_Rep = "[^a-zA-Z0-9]"
        Sequence_Pattern_Rep   = r"(.)\1\1+"
        Sequence_Replace_Pattern = r"\1\1"
        emojis_Replacements = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
                  ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
                  ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
                  ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
                  '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
                  '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
                  ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}
        t = t.lower()
        t = charRemoval.sub(urlLinks,' URL',t)
        for emoji in emojis_Replacements.keys():
            t = t.replace(emoji, "EMOJI" + emojis_Replacements[emoji]) 
        t = charRemoval.sub(username, ' USER', t)
        t = charRemoval.sub(Alpha_Patterns_Rep, " ", t)
        t = charRemoval.sub(Sequence_Pattern_Rep, Sequence_Replace_Pattern, t)
        wordcleaned=' '
        #loop through t and if the word is important then append it
        for w in t.split():
#             if w not in stopwordlist:
            if len(w)>1: wordcleaned += (w+' ')
#                 w = WordNetLemmatizer().lemmatize(w)      
        return wordcleaned

In [3]:
def load_models():
    # Load the vectoriser.
    file = open('vectoriser.pickle', 'rb')
    vectoriser = pickle.load(file)
    file.close()
    # Load the LR Model.
    file = open('LogisticRegressionModel.pickle', 'rb')
    LRmodel = pickle.load(file)
    file.close()
    
    return vectoriser, LRmodel

def predict(vectoriser, model, text):
    # Predict the sentiment
    processedtext =TweetsPreProcessing().TweetsProcessing(text)
    textdata = vectoriser.transform(processedtext)
    st = model.predict(textdata)
    sentiment=["Sentiment"]
    processedtext.insert(0, "Processed Tweets")
    for i in st:
        if i==0:
            sentiment.append("Negative")
        else:
            sentiment.append("Positive")
    return sentiment, processedtext

def loadtext(filename):
    data_set = pd.read_csv(filename, names=["author id", "created_at", "geo", "id", "lang", "like_count", "quote_count", "reply_count", "retweet_count", "source", "tweet"],  encoding="ISO-8859-1" )
    data_set = data_set[["author id", "created_at", "geo", "id", "lang", "like_count", "quote_count", "reply_count", "retweet_count", "source", "tweet"]]
    f11= list(data_set["tweet"])
    f11.pop(0)
    return f11

In [4]:
def add_col_to_csv(csvfile,fileout,new_list):
    with open(csvfile, 'r') as read_f, \
        open(fileout, 'w', newline='') as write_f:
        csv_reader = csv.reader(read_f)
        csv_writer = csv.writer(write_f)
        i = 0
        for row in csv_reader:
            row.append(new_list[i])
            csv_writer.writerow(row)
            i += 1 

In [6]:
datainput=["Vaccine.csv", "Pfizer.csv", "moderna.csv", "sinopharm.csv", "Johnson.csv", "Booster.csv", "antivax.csv"]
dataoutput=["VaccineSentiment.csv", "PfizerSentiment.csv", "modernaSentiment.csv", "sinopharmSentiment.csv", "JohnsonSentiment.csv", "BoosterSentiment.csv", "antivaxSentiment.csv"]
temp=["temp1.csv","temp2.csv","temp3.csv","temp4.csv","temp5.csv","temp6.csv","temp7.csv"]
vectoriser, LRmodel = load_models()
for i in range(7):
    text  = loadtext(datainput[i])
    sentiment, processedtext = predict(vectoriser, LRmodel, text)
    add_col_to_csv(datainput[i],temp[i],processedtext)
    add_col_to_csv(temp[i],dataoutput[i],sentiment)
    print("Done Applying: ", datainput[i])

/var/folders/lw/f14mp2yd7sq7v92vn6ktx97w0000gn/T/ipykernel_52124/312876201.py:6: DtypeWarning: Columns (0,3,5,6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  text  = loadtext(datainput[i])


Done Applying:  Vaccine.csv
Done Applying:  Pfizer.csv
Done Applying:  moderna.csv
Done Applying:  sinopharm.csv
Done Applying:  Johnson.csv
Done Applying:  Booster.csv
Done Applying:  antivax.csv
